In [ ]:
!pip install torchnmf
#Implement libraries
import torch
import numpy as np
from scipy import signal
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable as V
from sklearn.decomposition import NMF
import torchnmf

In [ ]:
#Implement Nonnegative Matrix Factorization
X = np.array([[1, 1], [2, 1], [3, 1], [4, 1], [5, 1], [6, 1]])
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(X)
H = model.components_

In [ ]:
X.shape

(6, 2)

In [ ]:
print(W)
print(W.shape)

[[0.         0.46243941]
 [0.56287245 0.37784392]
 [1.12677624 0.29266953]
 [1.69067649 0.20749713]
 [2.25458746 0.12231871]
 [2.81849147 0.0371442 ]]
(6, 2)


In [ ]:
H

array([[2.10033104, 0.32627035],
       [2.16343685, 2.16145298]])

In [ ]:
print(np.matmul(W,H))
print(X)

[[1.00045845 0.99954104]
 [1.99965992 1.00034045]
 [2.99977515 1.0002251 ]
 [3.99988724 1.00011289]
 [5.00000882 0.99999117]
 [6.00012424 0.99987563]]
[[1 1]
 [2 1]
 [3 1]
 [4 1]
 [5 1]
 [6 1]]


In [ ]:
W=torch.tensor([[1, 4], [2, 4], [3, 3], [3, 2], [4, 2], [6, 1]],dtype=torch.float32,requires_grad=True)
x=torch.tensor(X.astype(np.float32))
class AutoEncoder(nn.Module):
    def __init__(self,w=W):
        super(AutoEncoder, self).__init__()
        self.w=w
        #Encoder
        self.encoder=nn.Linear(6,2,bias=False)
        
        #Decoder 
        self.decoder=nn.Linear(2,6,bias=False)
        self.encoder.weight=nn.Parameter((torch.linalg.pinv(self.w)))
        self.decoder.weight=nn.Parameter(self.w)
        
    def forward(self,x):
      encode=torch.relu(self.encoder(x.T))
      decode=self.decoder(encode)
      return self.w, encode, decode.T


In [ ]:
model_1=AutoEncoder()
total_loss=[]
list_para=[]
list_h=[]
list_w=[]
optimizer=torch.optim.Adam(model_1.parameters(),lr=1e-4,weight_decay=1e-3,eps=1e-4, betas=(0.90,0.99))
for i in range(5000):
  w_pred, h_pred,x_pred=model_1(x)
  loss=torchnmf.metrics.kl_div(x_pred,x)
  if torch.isnan(loss):
    break
  else:
    total_loss.append(loss)
    list_para.append(x_pred)
    list_h.append(h_pred)
    list_w.append(w_pred)
    loss.backward()
    optimizer.step()

print('best loss: ',np.min(total_loss))
print('x to predict: \n',x)
print('best x_pred: \n',list_para[np.argmin(total_loss)])
print('best h_pred: \n',list_h[np.argmin(total_loss)])
print('best w_pred: \n',list_w[np.argmin(total_loss)])



best loss:  tensor(0.0302, grad_fn=<AddBackward0>)
x to predict: 
 tensor([[1., 1.],
        [2., 1.],
        [3., 1.],
        [4., 1.],
        [5., 1.],
        [6., 1.]])
best x_pred: 
 tensor([[1.0071, 0.9966],
        [1.8790, 1.0274],
        [2.9975, 0.9849],
        [3.7391, 1.0206],
        [4.8588, 1.0691],
        [6.3634, 1.0616]], grad_fn=<PermuteBackward>)
best h_pred: 
 tensor([[1.1230, 0.0000],
        [0.1628, 0.2060]], grad_fn=<ReluBackward0>)
best w_pred: 
 tensor([[0.8328, 3.9460],
        [1.5532, 3.4652],
        [2.4738, 2.4777],
        [3.5259, 2.4720],
        [4.5178, 1.5717],
        [5.4667, 0.6612]], requires_grad=True)
